In [134]:
import numpy as np
import queue

# Backtracking search for Hamiltonian cycle

representation [S, 0, 0, 0, 0, 0, S]
search tree, decide on movement each step
data structure? 

In [135]:
graph = {'a':['b', 'd'], 
         'b':['a', 'c', 'f'], 
         'c':['a', 'b', 'd', 'e'], 
         'd':['a', 'c', 'e'], 
         'e':['c', 'd', 'f'], 
         'f':['b', 'e']}

In [146]:
q = queue.LifoQueue()
ps1 = (['a',0,0,0,0,0,0], 0)
q.put(ps1)
while not q.empty():
#     print(q.get())
    (ps, l) = q.get()
    print(ps, l)
    at_node = ps[l]
    if l == len(ps)-2:# can it go back to the start?
        if 'a' in graph[at_node]:
            ps[-1]='a'
            print('solution found: ',ps)
        
    dest_tup = graph[at_node]
    for dest in dest_tup:
        if dest not in ps:
            next_sol = ps.copy() # copy without reference
            next_sol[l+1] = dest
            q.put((next_sol, l+1))


['a', 0, 0, 0, 0, 0, 0] 0
['a', 'd', 0, 0, 0, 0, 0] 1
['a', 'd', 'e', 0, 0, 0, 0] 2
['a', 'd', 'e', 'f', 0, 0, 0] 3
['a', 'd', 'e', 'f', 'b', 0, 0] 4
['a', 'd', 'e', 'f', 'b', 'c', 0] 5
solution found:  ['a', 'd', 'e', 'f', 'b', 'c', 'a']
['a', 'd', 'e', 'c', 0, 0, 0] 3
['a', 'd', 'e', 'c', 'b', 0, 0] 4
['a', 'd', 'e', 'c', 'b', 'f', 0] 5
['a', 'd', 'c', 0, 0, 0, 0] 2
['a', 'd', 'c', 'e', 0, 0, 0] 3
['a', 'd', 'c', 'e', 'f', 0, 0] 4
['a', 'd', 'c', 'e', 'f', 'b', 0] 5
solution found:  ['a', 'd', 'c', 'e', 'f', 'b', 'a']
['a', 'd', 'c', 'b', 0, 0, 0] 3
['a', 'd', 'c', 'b', 'f', 0, 0] 4
['a', 'd', 'c', 'b', 'f', 'e', 0] 5
['a', 'b', 0, 0, 0, 0, 0] 1
['a', 'b', 'f', 0, 0, 0, 0] 2
['a', 'b', 'f', 'e', 0, 0, 0] 3
['a', 'b', 'f', 'e', 'd', 0, 0] 4
['a', 'b', 'f', 'e', 'd', 'c', 0] 5
solution found:  ['a', 'b', 'f', 'e', 'd', 'c', 'a']
['a', 'b', 'f', 'e', 'c', 0, 0] 4
['a', 'b', 'f', 'e', 'c', 'd', 0] 5
solution found:  ['a', 'b', 'f', 'e', 'c', 'd', 'a']
['a', 'b', 'c', 0, 0, 0, 0] 2
['a', 

# Branch and Bound search

Need to chose upper bound.. if total space left times highest ratio of value to weight left out means less value than already got, then no point continuing
could also try number of spaces left times by 

state representation? - [0,1,0,1,0,1,0,1,0,0] - 1 means item is included

In [137]:
def get_wv(knapsack, vals, weights):
    tw = knapsack.dot(weights)
    tv = knapsack.dot(vals)
    return tw, tv

def calc_ratio_ub(knapsack, vals, weights, b, ratios, max_weight):
    tw, tv = get_wv(knapsack, vals, weights)
    left = np.zeros(10)
    left[b+1:] = 1- knapsack[b+1:]

    twleft = max_weight - tw # finds total weight left
    rleft = np.multiply(ratios, left) # finds ratios left
    maxr = np.argmax(rleft) # gives index of largest ratio left
    best_poss_val = ratios[maxr]*twleft+ tv  # gives best possible weight from this partial solution
    return best_poss_val

def calc_items_ub(knapsack, vals, weights, b, ratios, max_weight):
    tw, tv = get_wv(knapsack, vals, weights)
    left = np.zeros(10)
    n_items_left = 10-b-1
    left[b+1:] = 1- knapsack[b+1:]

    items_left = (b+1)-len(ratios)
    rleft = np.multiply(vals, left) # finds values left
    maxr = np.argmax(rleft) # gives index of largest value left
    best_poss_val = vals[maxr]*(n_items_left)+ tv  # gives best possible value from this partial solution
    return best_poss_val


In [138]:


def knapsack(vals, weights, max_weight, calc_ub_fn):
    ratios = np.divide(vals, weights)
    q = queue.LifoQueue()
    best_val = 0

    first = np.array([0,0,0,0,0,0,0,0,0,0])
    second = np.array([1,0,0,0,0,0,0,0,0,0])
    max_b = len(first)
    q.put((first, 0))
    q.put((second, 0))
    evals = 0
    while not q.empty():
        evals+=1
        ps, b = q.get()
        tw, tv = get_wv(ps, vals, weights)
#         print(ps, 'level', b, 'value', tv, 'weight', tw, 'upper bound', ub, max_b-1)

        if tw>max_weight:#overfull. stop here. use next from queue. 
#             print('too full')
            continue

        if b==max_b-1:#is a full and valid solution.
            if tv > best_val:
                print('best val found', tv, ps)
                best_val = tv
                best_sol=ps
            continue

        ub = calc_ub_fn(ps, vals, weights, b, ratios, max_weight)
        if ub < best_val: # branch and bound
#             print('upper bound too low')
            # if solution cannot possibly give a good answer then look at the next in the queue
            continue

        # add next ones to the queue
        nx = np.copy(ps) #copy without ref
        q.put((nx, b+1)) # the same but on the next level
        ny = np.copy(ps)
        ny[b+1]=1
        q.put((ny, b+1))

    return best_sol, best_val, evals


In [139]:
vals = np.array([100,4,30,60,200,95,5,10,20,12])
weights = np.array([24, 14, 39, 60, 98, 22, 5, 32, 16, 9])

best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_ratio_ub)

print('best solution', best_sol, best_val, 'evals = ', evals)

best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_items_ub)

print('best solution', best_sol, best_val, 'evals = ', evals)  

best val found 199 [1 1 1 1 0 0 1 0 0 0]
best val found 206 [1 1 1 1 0 0 0 0 0 1]
best val found 256 [1 1 1 0 0 1 1 1 0 1]
best val found 266 [1 1 1 0 0 1 1 0 1 1]
best val found 296 [1 1 0 1 0 1 1 0 1 1]
best val found 321 [1 1 0 0 1 0 1 0 0 1]
best val found 400 [1 0 0 0 1 1 1 0 0 0]
best solution [1 0 0 0 1 1 1 0 0 0] 400 evals =  166
best val found 199 [1 1 1 1 0 0 1 0 0 0]
best val found 206 [1 1 1 1 0 0 0 0 0 1]
best val found 256 [1 1 1 0 0 1 1 1 0 1]
best val found 266 [1 1 1 0 0 1 1 0 1 1]
best val found 296 [1 1 0 1 0 1 1 0 1 1]
best val found 321 [1 1 0 0 1 0 1 0 0 1]
best val found 400 [1 0 0 0 1 1 1 0 0 0]
best solution [1 0 0 0 1 1 1 0 0 0] 400 evals =  188


# Sort by inverse weight?

In [144]:
vals = np.array([100,4,30,60,200,95,5,10,20,12])
weights = np.array([24, 14, 39, 60, 98, 22, 5, 32, 16, 9])
ratios = np.divide(vals, weights)
vw = list(zip(vals, weights, ratios))

vw.sort(key=lambda y: y[1])
vals, weights, ratios = zip(*vw)

In [145]:
best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_ratio_ub)
print('best solution', best_sol, best_val, 'evals = ', evals)

best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_items_ub)
print('best solution', best_sol, best_val, 'evals = ', evals) 

best val found 246 [1 1 1 1 1 1 1 0 0 0]
best val found 266 [1 1 1 1 1 1 0 1 0 0]
best val found 296 [1 1 1 1 1 1 0 0 1 0]
best val found 316 [1 1 1 0 1 0 0 0 0 1]
best val found 321 [1 1 1 0 0 1 0 0 0 1]
best val found 332 [1 1 0 1 1 0 0 0 0 1]
best val found 400 [1 0 0 0 1 1 0 0 0 1]
best solution [1 0 0 0 1 1 0 0 0 1] 400 evals =  354
best val found 246 [1 1 1 1 1 1 1 0 0 0]
best val found 266 [1 1 1 1 1 1 0 1 0 0]
best val found 296 [1 1 1 1 1 1 0 0 1 0]
best val found 316 [1 1 1 0 1 0 0 0 0 1]
best val found 321 [1 1 1 0 0 1 0 0 0 1]
best val found 332 [1 1 0 1 1 0 0 0 0 1]
best val found 400 [1 0 0 0 1 1 0 0 0 1]
best solution [1 0 0 0 1 1 0 0 0 1] 400 evals =  1348


## Sort by value?

In [142]:
vals = np.array([100,4,30,60,200,95,5,10,20,12])
weights = np.array([24, 14, 39, 60, 98, 22, 5, 32, 16, 9])
ratios = np.divide(vals, weights)
vw = list(zip(vals, weights, ratios))

vw.sort(key=lambda y: y[0], reverse=True)
vals, weights, ratios = zip(*vw)

In [143]:
best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_ratio_ub)

print('best solution', best_sol, best_val, 'evals = ', evals)

best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_items_ub)

print('best solution', best_sol, best_val, 'evals = ', evals) 

best val found 400 [1 1 1 0 0 0 0 0 1 0]
best solution [1 1 1 0 0 0 0 0 1 0] 400 evals =  28
best val found 400 [1 1 1 0 0 0 0 0 1 0]
best solution [1 1 1 0 0 0 0 0 1 0] 400 evals =  50


# Possible improvements: sort by ratio?

In [140]:
vals = np.array([100,4,30,60,200,95,5,10,20,12])
weights = np.array([24, 14, 39, 60, 98, 22, 5, 32, 16, 9])
ratios = np.divide(vals, weights)
vw = list(zip(vals, weights, ratios))

vw.sort(key=lambda y: y[2], reverse=True)
vals, weights, ratios = zip(*vw)

In [141]:
best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_ratio_ub)

print('best solution', best_sol, best_val, 'evals = ', evals)

best_sol, best_val, evals = knapsack(vals, weights, max_weight, calc_items_ub)

print('best solution', best_sol, best_val, 'evals = ', evals)  

best val found 400 [1 1 1 0 0 0 1 0 0 0]
best solution [1 1 1 0 0 0 1 0 0 0] 400 evals =  22
best val found 400 [1 1 1 0 0 0 1 0 0 0]
best solution [1 1 1 0 0 0 1 0 0 0] 400 evals =  154


# Genetic Algorithm to solve Knapsack

In [154]:
# randomly generate starting population
N = 100
l = 10
pop = []
for i in range(N):
    sol = []
    for j in range(l):
        if np.random.rand()>0.5:
            sol.append(0)
        else:
            sol.append(1)
    pop.append(np.array(sol))

In [155]:
pop

[array([1, 0, 0, 0, 0, 1, 0, 0, 1, 0]),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 1]),
 array([0, 0, 1, 1, 0, 0, 1, 0, 1, 0]),
 array([1, 0, 1, 0, 1, 0, 1, 1, 1, 1]),
 array([1, 0, 1, 1, 0, 0, 0, 1, 0, 0]),
 array([1, 1, 1, 1, 0, 0, 1, 0, 1, 1]),
 array([1, 1, 1, 0, 0, 1, 1, 0, 0, 0]),
 array([1, 0, 0, 0, 1, 1, 0, 1, 1, 0]),
 array([1, 1, 1, 0, 0, 1, 0, 1, 1, 1]),
 array([0, 0, 0, 1, 0, 1, 1, 0, 0, 1]),
 array([1, 0, 0, 0, 1, 0, 0, 1, 0, 1]),
 array([1, 1, 0, 1, 0, 0, 1, 0, 1, 1]),
 array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0]),
 array([1, 0, 0, 1, 0, 0, 1, 0, 1, 0]),
 array([0, 1, 1, 0, 0, 1, 1, 1, 0, 1]),
 array([0, 1, 0, 1, 0, 1, 0, 0, 0, 1]),
 array([0, 0, 1, 1, 0, 1, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 1, 0, 1, 0, 1, 0]),
 array([0, 1, 1, 1, 1, 0, 1, 1, 0, 1]),
 array([0, 0, 0, 0, 1, 0, 1, 1, 0, 0]),
 array([0, 1, 0, 0, 1, 1, 1, 0, 0, 1]),
 array([1, 0, 0, 1, 1, 0, 1, 0, 1, 1]),
 array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1]),
 array([1, 0, 1, 0, 0, 1, 0, 0, 0, 0]),


In [156]:
#evaluate population

def fitness(sol, vals, weights):
    tv, tw = get_wv(sol, vals, weights)
    if tw<=150:
        return tv
    return 0

def mutate(sol, mut_prob):
    if np.random.rand()<mut_prob:
        i = np.random.choice(10)
        sol[i]=1-sol[i]
    return sol

mut_prob = 0.1
for t in range(100):
    #evaluate
    fits = []
    for sol in pop:
        fits.append(fitness(sol, vals, weights))
    fits = np.array(fits)
    print('max fitness is: ', max(fits), pop[np.argmax(fits)])
    #select
    probs = fits/sum(fits)
    print(probs)
    new_pop = []
    for i in range(N):
        ind = np.random.choice(N, p=probs)
        new_sol = pop[ind]
        #mutate
        new_sol = mutate(new_sol, mut_prob)
        new_pop.append(new_sol)
    # start again with new population
    pop = new_pop
        

max fitness is:  159 [1 1 1 0 0 0 1 1 1 0]
[0.         0.         0.05781991 0.         0.03507109 0.
 0.03981043 0.         0.         0.         0.         0.
 0.         0.0535545  0.         0.         0.02559242 0.
 0.         0.04407583 0.         0.         0.01279621 0.
 0.02037915 0.03459716 0.         0.         0.02511848 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.00663507 0.06018957
 0.         0.         0.         0.         0.05450237 0.
 0.01042654 0.         0.         0.         0.         0.
 0.         0.         0.         0.03412322 0.         0.
 0.         0.         0.         0.         0.06777251 0.01279621
 0.         0.         0.01706161 0.         0.         0.
 0.         0.02559242 0.03601896 0.05781991 0.         0.03554502
 0.         0.         0.         0.         0.0492891  0.03222749
 0.         0.         0.02227488 0.         0.         0.
 0.         0.07535545 0.         0.    

/var/folders/dl/pczkvbw15215r_302bklrz2m0000gq/T/ipykernel_79183/3419066650.py:24: RuntimeWarning: invalid value encountered in true_divide
  probs = fits/sum(fits)


ValueError: probabilities contain NaN